## chap05 ニューラルネットワーク

1.   データが置かれたGoogleドライブをマウント（Colabのメニューで操作）


> ファイル -> ドライブをマウント -> 実行可能なセルが現れる -> ログインして認証コードを入力




2.   /content/driveにマウントされる



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


GiNZAのインストール  

In [2]:
!pip install -U ginza

Requirement already up-to-date: ginza in /usr/local/lib/python3.6/dist-packages (3.1.2)


Lightningのインストール

In [3]:
!pip install pytorch-lightning
import pkg_resources, imp
imp.reload(pkg_resources)

<module 'pkg_resources' from '/usr/local/lib/python3.6/dist-packages/pkg_resources/__init__.py'>

GiNZAの使用準備

In [0]:
import spacy
nlp = spacy.load('ja_ginza')

京都大学情報学研究科--NTTコミュニケーション科学基礎研究所 共同研究ユニット 
 [解析済みブログコーパス](http://nlp.ist.i.kyoto-u.ac.jp/kuntt/#ga739fe2)
の読み込み

In [0]:
Dpath = '/content/drive/My Drive/Resources/KNBC_v1.0_090925_utf8/corpus2/'
topics = ['Gourmet', 'Keitai', 'Kyoto', 'Sports']

In [0]:
import pandas as pd
import re

cols = ['doc', 'label']
df = pd.DataFrame(index=[], columns=cols)
p = re.compile(r'(［.*］)(.*)')

for topic in topics:
  data = pd.read_table(Dpath+topic+'.tsv', header=None, dtype=str, usecols=[1])
  for d in data.itertuples():
    m = p.match(d[1])
    if m != None:  # first line of the document
      if d[0] != 0:
        df = df.append(pd.DataFrame([doc.strip(), topic], index = cols).T)
      sent = m.groups()[1]
      doc = ''
    else:
      sent = d[1]
    doc = doc + ' ' +  ' '.join([t.text for t in nlp(sent)])
  df = df.append(pd.DataFrame([doc.strip(), topic], index = cols).T)

In [7]:
df

,doc,label
0,烏丸 六角 の おかき 屋 さん 六角堂 の 前 に ある 、 蕪村 庵 と いう お店 に...,Gourmet
0,河原町 の 居酒屋 この 間 先輩 たち に つれ られ 、 河原町 の 居酒屋 へ 行っ ...,Gourmet
0,ちょっと 贅沢 ほんの ちょこっと な ん だ けど 、 贅沢 し たい とき に 何 を ...,Gourmet
0,食欲 の 秋 最近 おなか が 減っ て 仕方 あり ませ ん か 。 仕方 あり ませ ん...,Gourmet
0,季節 限定 も のっ て … １０ 月 と いえ ば 秋 本番 で ある 。 栗 や 秋刀魚...,Gourmet
...,...,...
0,クライマックス シリーズ 今年 の 阪神 は なん やっ た ん や ！ ！ 初め 弱く て...,Sports
0,京都 サンガ に つい て この ブログ の 右側 に 検索 窓 ある じゃ ない です か...,Sports
0,サッカー から 伺える ルール と の 付き合い 方 私自身 は 全く の サッカー 素人 ...,Sports
0,する か 見る か スポーツ は する 派 です か ？ し たい 見る 派 です か ？ ...,Sports


In [8]:
df['label'].value_counts(sort=False)

Sports     22
Kyoto      91
Keitai     79
Gourmet    57
Name: label, dtype: int64

# 文書分類

## 学習データの作成

文章をtfidfベクトルへ

In [0]:
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [0]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(df.values[:,1])
X = df.values[:,0]

In [0]:
vectorizer = TfidfVectorizer()
Xv = vectorizer.fit_transform(X)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(Xv, y, test_size=0.3, random_state=1)

In [0]:
Xtrain = torch.Tensor(X_train.todense())
ytrain = torch.LongTensor(y_train)
Xtest = torch.Tensor(X_test.todense())
ytest = torch.LongTensor(y_test)
train = torch.utils.data.TensorDataset(Xtrain, ytrain)
test = torch.utils.data.TensorDataset(Xtest, ytest)

## 3層ネットワーク

In [0]:
input_size = X_train.shape[1]
output_size = len(topics)

class Net(pl.LightningModule):
    def __init__(self, input_size=input_size, hidden_size=10, output_size=output_size, batch_size=10):
        super(Net, self).__init__()
        self.L1 = nn.Linear(input_size, hidden_size)
        self.L2 = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
        
    def forward(self, x):
        x = self.L1(x)
        x = F.relu(x)
        x = self.L2(x)
        return x
    
    def lossfun(self, y, t):
        return F.cross_entropy(y, t)
    
    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.1)
        
    @pl.data_loader
    def train_dataloader(self):
        return torch.utils.data.DataLoader(train, self.batch_size, shuffle=True, num_workers=4)

    def test_dataloader(self):
        # can also return a list of test dataloaders
        return torch.utils.data.DataLoader(test, self.batch_size, shuffle=False, num_workers=4)

    def training_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        results = {'loss': loss}
        return results

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        return {'test_loss': F.cross_entropy(y_hat, y)}

    def test_epoch_end(self, outputs):
        test_loss_mean = torch.stack([x['test_loss'] for x in outputs]).mean()
        return {'test_loss': test_loss_mean}

In [15]:
net = Net()
trainer = Trainer(max_nb_epochs=50)
trainer.fit(net)
trainer.test()

INFO:lightning:GPU available: False, used: False
INFO:lightning:
  | Name | Type   | Params
----------------------------
0 | L1   | Linear | 71 K  
1 | L2   | Linear | 44    


--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(0.6075)}
--------------------------------------------------------------------------------



テストデータでの評価

In [16]:
y_pred = []

with torch.no_grad():
    for (docs, labels) in torch.utils.data.DataLoader(test, 1, shuffle=False):
        outputs = net(docs)
        _, predicted = torch.max(outputs.data, 1)
        y_pred.append(predicted.item())

print(classification_report(y_test, y_pred, target_names=topics))

              precision    recall  f1-score   support

     Gourmet       0.75      0.95      0.84        19
      Keitai       1.00      0.90      0.95        21
       Kyoto       0.86      0.92      0.89        26
      Sports       1.00      0.44      0.62         9

    accuracy                           0.87        75
   macro avg       0.90      0.80      0.82        75
weighted avg       0.89      0.87      0.86        75

